In [1]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [3]:
url = "https://www.transfermarkt.es/valencia-cf/kader/verein/1049/plus/0/galerie/0?saison_id=2016"

In [4]:
plantilla17 = requests.get(url, headers=headers)


In [5]:
plantilla17 = BeautifulSoup(plantilla17.text, "html.parser")

In [6]:
temp17={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [7]:
for valor in plantilla17.find_all("td", class_ = "rechts hauptlink"):
    temp17["Valor"].append(valor.text)

SACAR NOMBRE JUGADOR

In [8]:
nombres = plantilla17.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp17["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [9]:
edad = plantilla17.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp17["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [10]:
posiciones = plantilla17.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp17["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [11]:
nacionalidades = plantilla17.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img", class_="flaggenrahmen")
    if img:
        #print(img[0].get("title"))
        temp17["Nacionalidad"].append(img[0].get("title"))

In [12]:
temporada17 =  pd.DataFrame(temp17)

In [13]:
temporada17.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Diego Alves,Portero,32,Brasil,"5,00 mill. €"
1,Mathew Ryan,Portero,25,Australia,"3,50 mill. €"
2,Jaume Doménech,Portero,26,España,"2,50 mill. €"
3,Shkodran Mustafi,Defensa central,25,Alemania,"30,00 mill. €"
4,Eliaquim Mangala,Defensa central,26,Francia,"15,00 mill. €"
5,Ezequiel Garay,Defensa central,30,Argentina,"15,00 mill. €"
6,Aymen Abdennour,Defensa central,27,Túnez,"6,00 mill. €"
7,Aderllan Santos,Defensa central,28,Brasil,"3,50 mill. €"
8,Rúben Vezo,Defensa central,23,Portugal,"1,50 mill. €"
9,Javi Jiménez,Defensa central,20,España,300 mil €


In [14]:
temporada17.Edad = temporada17.Edad.astype(int)

In [15]:
temporada17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Nombre        38 non-null     object
 1   Posicion      38 non-null     object
 2   Edad          38 non-null     int32 
 3   Nacionalidad  38 non-null     object
 4   Valor         38 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.5+ KB


In [16]:
temporada17.Valor = temporada17.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_21976\1979096239.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada17.Valor = temporada17.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [17]:
temporada17.loc[temporada17.Valor >= 100,"Valor"] = temporada17.loc[temporada17.Valor >= 100,"Valor"]/1000


In [18]:
temporada17["Valor millones"] = temporada17.Valor 

In [19]:
temporada17 = temporada17.drop(columns=["Valor"])

In [20]:
temporada17

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Diego Alves,Portero,32,Brasil,5.0
1,Mathew Ryan,Portero,25,Australia,3.5
2,Jaume Doménech,Portero,26,España,2.5
3,Shkodran Mustafi,Defensa central,25,Alemania,30.0
4,Eliaquim Mangala,Defensa central,26,Francia,15.0
5,Ezequiel Garay,Defensa central,30,Argentina,15.0
6,Aymen Abdennour,Defensa central,27,Túnez,6.0
7,Aderllan Santos,Defensa central,28,Brasil,3.5
8,Rúben Vezo,Defensa central,23,Portugal,1.5
9,Javi Jiménez,Defensa central,20,España,0.3


In [21]:
#temporada22["Valor millones"].fillna(temporada22["Valor millones"].min, inplace=True)
min_valor = temporada17["Valor millones"].min()
temporada17["Valor millones"].fillna(min_valor, inplace=True)

C:\Users\raul_\AppData\Local\Temp\ipykernel_21976\2835379363.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temporada17["Valor millones"].fillna(min_valor, inplace=True)


In [22]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada17_16.csv"

temporada17.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_21976\3518012886.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada17_16.csv"
